## Extract administrative region population

In [109]:
import json
import sys
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pandas as pd

In [110]:
# file path
data_path = '../data/raw/administrative_regions_Trentino.json'

In [111]:
# load the data
with open(data_path) as json_file:
    data = json.load(json_file)

In [112]:
data.keys()

dict_keys(['count', 'items', 'datagem-version'])

In [115]:
# extract features
data_features = gpd.GeoDataFrame(data['items'])

In [117]:
data_features

,euroCode,website,population,cadastralCode,elevation,acheneID,name,level,geometry,provenance,wikipedia,isProvinceCheflieu,postCodes,parentAchenes,alternateNames,localCode,isMountainMunicipality,geomComplex
0,None,http://www.comune.sanmichelealladige.tn.it,"{'2001': None, '2011': 2902}",I042,228,http://dandelion.eu/resource/32ca86fe3c8f5988a...,San Michele all'Adige,60,"{'type': 'Polygon', 'coordinates': [[[11.13966...","[Geonames, ISTAT, SpazioDati, SpazioDati Partn...",{'de': 'http://de.wikipedia.org/wiki/San_Miche...,False,[38010],{'province': 'http://dandelion.eu/resource/c67...,"[Comune di San Michele all'Adige, San Michele ...",022167,T,"{'provenance': 0, 'accuracy': 100}"
1,None,http://www.comune.roncegnoterme.tn.it/,"{'2001': None, '2011': 2814}",H528,535,http://dandelion.eu/resource/629cb36715565daba...,Roncegno Terme,60,"{'type': 'Polygon', 'coordinates': [[[11.38799...","[Geonames, ISTAT, SpazioDati, SpazioDati Partn...",{'de': 'http://de.wikipedia.org/wiki/Roncegno'...,False,[38050],{'province': 'http://dandelion.eu/resource/c67...,"[Comune di Roncegno Terme, Roncegno]",022156,T,"{'provenance': 0, 'accuracy': 100}"
2,None,None,"{'2001': None, '2011': 254}",D250,600,http://dandelion.eu/resource/b7d989146d7440ef5...,Darè,60,"{'type': 'Polygon', 'coordinates': [[[10.71919...","[Geonames, ISTAT, SpazioDati, SpazioDati Partn...",{'de': 'http://de.wikipedia.org/wiki/Dar%C3%A8...,False,[38080],{'province': 'http://dandelion.eu/resource/c67...,"[Comune di Dare, Comune di Darè, Dare, Dare']",022073,T,"{'provenance': 0, 'accuracy': 100}"
3,None,http://www.comune.zambana.tn.it/,"{'2001': None, '2011': 1620}",M142,206,http://dandelion.eu/resource/129d7ed405be31701...,Zambana,60,"{'type': 'MultiPolygon', 'coordinates': [[[[11...","[Geonames, ISTAT, SpazioDati, SpazioDati Partn...","{'de': 'http://de.wikipedia.org/wiki/Zambana',...",False,[38010],{'province': 'http://dandelion.eu/resource/c67...,[Comune di Zambana],022225,T,"{'provenance': 0, 'accuracy': 100}"
4,None,http://www.comune.flavon.tn.it,"{'2001': None, '2011': 503}",D631,575,http://dandelion.eu/resource/36ab052b42b2e822b...,Flavon,60,"{'type': 'Polygon', 'coordinates': [[[11.04318...","[Geonames, ISTAT, SpazioDati, SpazioDati Partn...",{'de': 'http://de.wikipedia.org/wiki/Flavon_%2...,False,[38010],{'province': 'http://dandelion.eu/resource/c67...,[Comune di Flavon],022086,T,"{'provenance': 0, 'accuracy': 100}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181,None,None,"{'2001': 24, '2011': None}",None,1319,http://dandelion.eu/resource/258a0dda39ed3a58e...,Nocchi,70,None,[ISTAT],"{'de': None, 'en': None, 'it': None}",None,[],{'province': 'http://dandelion.eu/resource/c67...,[],02208720015,None,"{'provenance': None, 'accuracy': None}"
1182,None,None,"{'2001': 158, '2011': None}",None,1395,http://dandelion.eu/resource/87d755a6d75b4c93b...,Fontanazzo,70,None,[ISTAT],"{'de': None, 'en': None, 'it': None}",None,[],{'province': 'http://dandelion.eu/resource/c67...,[],02211310001,None,"{'provenance': None, 'accuracy': None}"
1183,None,None,"{'2001': 270, '2011': None}",None,1129,http://dandelion.eu/resource/e064a6f9218ca3017...,Montesover,70,None,[ISTAT],"{'de': None, 'en': None, 'it': None}",None,[],{'province': 'http://dandelion.eu/resource/c67...,[],02217710001,None,"{'provenance': None, 'accuracy': None}"
1184,None,None,"{'2001': 334, '2011': None}",None,1123,http://dandelion.eu/resource/8aeea47ee6ae193ed...,Sant'Antonio di Mavignola,70,None,"[ISTAT, SpazioDati, Wikipedia in Italiano]","{'de': None, 'en': None, 'it': 'http://it.wiki...",None,[38086],{'province': 'http://dandelion.eu/resource/c67...,[],02214310004,None,"{'provenance': None, 'accuracy': None}"


In [129]:
df = data_features[['population', 'elevation', 'name']]

In [130]:
df

,population,elevation,name
0,"{'2001': None, '2011': 2902}",228,San Michele all'Adige
1,"{'2001': None, '2011': 2814}",535,Roncegno Terme
2,"{'2001': None, '2011': 254}",600,Darè
3,"{'2001': None, '2011': 1620}",206,Zambana
4,"{'2001': None, '2011': 503}",575,Flavon
...,...,...,...
1181,"{'2001': 24, '2011': None}",1319,Nocchi
1182,"{'2001': 158, '2011': None}",1395,Fontanazzo
1183,"{'2001': 270, '2011': None}",1129,Montesover
1184,"{'2001': 334, '2011': None}",1123,Sant'Antonio di Mavignola


In [131]:
type(df['population'].iloc[0])

dict

In [132]:
# Normalize the 'population' column, creating separate columns for each year
population_df = pd.json_normalize(df['population'])

# Rename the columns for clarity if needed
population_df.columns = ['population_2001', 'population_2011']

# Concatenate the extracted population columns with the original DataFrame
df = pd.concat([df, population_df], axis=1)

# Merge the 'population_2001' and 'population_2011' columns, giving preference to 'population_2011' if both exist
df['population'] = df['population_2011'].combine_first(df['population_2001']).astype(int)

# Now check if population_merged came from population_2011 (not null in 2011)
df['has_2011'] = df['population_2011'].notna()

# Drop duplicates, prioritizing rows with population from 2011
df = df.sort_values(by='has_2011', ascending=False).drop_duplicates(subset='name', keep='first')

# Drop the helper column 'has_2011' and the no longer needed population columns
df = df.drop(columns=['has_2011', 'population_2001', 'population_2011'])

# Display the resulting DataFrame
df

,population,elevation,name
0,2902,228,San Michele all'Adige
182,1893,456,Terlago
166,928,640,Sanzeno
167,507,710,Fiera di Primiero
169,999,608,Villa Rendena
...,...,...,...
542,2339,228,San Michele All'Adige
543,101,1148,Termenago
544,15,1136,Mezzaselva
545,10,1400,Tolleri


In [133]:
df[df['name'] == 'Trento']

,population,elevation,name
22,114063,194,Trento


In [134]:
df.to_csv('../data/interim/adm_regions.csv')